# Demo: loading data from Amazon into iguazio DB

In [1]:
import pandas as pd
import v3io_frames as v3f
client = v3f.Client()
tablename = 'bank'

## Step 1: Load data from Amazon S3 into Pandas

In [2]:
# read S3 file into a data frame and show its data & metadata
df = pd.read_csv('https://s3.amazonaws.com/apache-zeppelin/tutorial/bank/bank.csv', sep=';')
df.head()

age          job  marital  education default  balance housing loan  \
0   30   unemployed  married    primary      no     1787      no   no   
1   33     services  married  secondary      no     4789     yes  yes   
2   35   management   single   tertiary      no     1350     yes   no   
3   30   management  married   tertiary      no     1476     yes  yes   
4   59  blue-collar  married  secondary      no        0     yes   no   

    contact  day month  duration  campaign  pdays  previous poutcome   y  
0  cellular   19   oct        79         1     -1         0  unknown  no  
1  cellular   11   may       220         1    339         4  failure  no  
2  cellular   16   apr       185         1    330         1  failure  no  
3   unknown    3   jun       199         4     -1         0  unknown  no  
4   unknown    5   may       226         1     -1         0  unknown  no

## Step 2: Write Pandas dataframe into the database using a single command
data is streamed into the database via fast NoSQL APIs

In [3]:
out = client.write('v3io', tablename, df)

## Step 3: Read from the Database with DB side SQL
offload data filtering, grouping, joins, etc to a scale-out high speed DB engine.
Using Apache Presto as the ANSI SQL processing layer over iguazio NoSQL

In [4]:
%sql select * from bigdata.bank where balance > 10000

Done.


[('no', 'secondary', 0, 'no', 'unknown', 219, 'married', 'no', 26452, 'jul', 'telephone', 2, 'no', 'retired', 15, 75, -1),
 ('no', 'secondary', 0, 'yes', 'unknown', 249, 'married', 'no', 19317, 'aug', 'cellular', 1, 'yes', 'retired', 4, 68, -1)]

## Read the data as a structured stream
the frames API returns dataframe iterator (a stream) to accelerate performance
and allow cuncurent data movement and processing

In [5]:
df = pd.concat(client.read(backend='v3io', table=tablename, filter="balance>20000"), sort=False)
df.head(8)

contact  day poutcome default  age __name  duration  education loan  \
__name                                                                          
75      telephone   15  unknown      no   75     75       219  secondary   no   

        balance housing month  previous      job   y  campaign  pdays  marital  
__name                                                                          
75        26452      no   jul         0  retired  no         2     -1  married